In [79]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from numpy import random



In [112]:
random.seed(42)

data = pd.read_csv('data/Features_For_Traditional_ML_Techniques.csv')
#remove all meta-data features as we are comparing structure (except hashtags)
#compare to original data performance on baseline models
meta_features = ["followers_count", "friends_count","favourites_count","statuses_count","listed_count","following","embeddings","BotScore",	"BotScoreBinary","cred","normalize_influence","mentions","quotes","replies","retweets","favourites","URLs"]
#filter data to only include non-meta features
data = data.drop(meta_features, axis=1)

grouped = data.groupby('statement')
train_data = pd.DataFrame()
test_data = pd.DataFrame()
i=0

for group_name, group_df in grouped:
    if random.random() <= 0.8:
        train_data = pd.concat([train_data, group_df])
    else:
        test_data = pd.concat([test_data, group_df])
    i += 1
    # if i > 100:
    #     break

# Step 2: Split each group into training and test sets

# for group_name, group_df in grouped:
#     if len(group_df) > 1:  # Ensure at least 2 samples for splitting
#         train_group, test_group = train_test_split(group_df, test_size=0.2, random_state=42)
#         train_data = pd.concat([train_data, train_group])
#         test_data = pd.concat([test_data, test_group])


#TEMP drop tweets + statements
train_data = train_data.drop(["tweet", "statement"], axis=1)
test_data = test_data.drop(["tweet", "statement"], axis=1)

#split into training_testing data with 80% statements in training and 20% in testing

# data.head()

In [113]:
len(train_data), len(test_data)

(106733, 27465)

In [114]:
train_targets= train_data['BinaryNumTarget']
train_data_notarget = train_data.drop(['BinaryNumTarget'], axis=1)

test_targets= test_data['BinaryNumTarget']
test_data_notarget = test_data.drop(['BinaryNumTarget'], axis=1)

print(train_targets.value_counts())

BinaryNumTarget
1.0    55988
0.0    50745
Name: count, dtype: int64


In [115]:
#fit random forest model
rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(train_data_notarget, train_targets)



RandomForestClassifier(random_state=0)

In [117]:
rf.predict(test_data_notarget)
print(f1_score(test_targets, rf.predict(test_data_notarget)))
print(accuracy_score(test_targets, rf.predict(test_data_notarget)))

0.9782056218713902
0.9793919533952303
